## Socket Server

[Python simple socket client/server using asyncio](https://stackoverflow.com/questions/48506460/python-simple-socket-client-server-using-asyncio)

[Socket Programming in Python](https://realpython.com/python-sockets/#echo-client-and-server)

## One Client and One Server

The Server can only accept and handle one Client at the same time. For example, the second Client will be blocked after the previous Client close its connection.

In [ ]:
# Server
import socket

HOST = ""  # Symbolic name meaning all available interfaces
PORT = 50007  # Arbitrary non-privileged port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)

    try:
        while True:
            client, addr = s.accept()
            print(f"Connected by client[{addr}]")
            while True:
                data = client.recv(1024)
                print(
                    f"raw_data[{data}] size[{len(data) if data else -1}] type[{type(data)}]"
                )
                if (
                    (not data)
                    or (data == b"\n")
                    or (data == b"quit\n")
                    or (data == b"close\n")
                ):
                    # client console type `Enter` to close client
                    # client console type `close` to shutdown server
                    break
                client.sendall(data)
            print(f"Disconnected by {client.getpeername()}")
            client.close()
            if data == b"close\n":
                break

    except KeyboardInterrupt:
        # exited in notebook via "Kernel --> Interrupt"
        # exited in console by "CTRL+C"
        print(f"gracefully close")

# Client
# nc 127.0.0.1 50007
# > hello mom
# > quit
# > close


## Multiple Clients and One Server(Multiple Threading)

In [ ]:
import threading
import socket

HOST = ""  # Symbolic name meaning all available interfaces
PORT = 50007  # Arbitrary non-privileged port

def handle_client(client):
    data = None
    while True:
        data = client.recv(255)
        print(
            f"raw_data[{data}] size[{len(data) if data else -1}] type[{type(data)}]"
        )
        if (
            (not data)
            or (data == b"\n")
            or (data == b"quit\n")
            or (data == b"quit")
        ):
            # client console type `Enter` to close client
            break
        client.sendall(data)
    print(f"Disconnected by {client.getpeername()}")
    client.close()

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(8)

try:
    while True:
        client, addr = server.accept()
        print(f"Connected by client[{addr}]")
        threading.Thread(target=handle_client, args=(client,)).start()
except KeyboardInterrupt:
    print(f"gracefully close")
    server.close()


# Client 1
# nc 127.0.0.1 50007
# > hello mom 1
# > quit

# Client 2
# nc 127.0.0.1 50007
# > hello mom 2
# > quit

## Multiple Clients and One Server(asyncio)

In [ ]:
import asyncio, socket

HOST = ""  # Symbolic name meaning all available interfaces
PORT = 50007  # Arbitrary non-privileged port

async def handle_client(client):
    loop = asyncio.get_event_loop()
    while True:
        data = await loop.sock_recv(client, 255)
        print(
            f"raw_data[{data}] size[{len(data) if data else -1}] type[{type(data)}]"
        )
        if (
            (not data)
            or (data == b"\n")
            or (data == b"quit\n")
            or (data == b"quit")
        ):
            # client console type `Enter` to close client
            break

        await loop.sock_sendall(client, data)
    print(f"Disconnected by {client.getpeername()}")
    client.close()

async def run_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen(8)
    server.setblocking(False)

    loop = asyncio.get_event_loop()

    while True:
        client, addr = await loop.sock_accept(server)
        print(f"Connected by client[{addr}]")
        loop.create_task(handle_client(client))

# Jupyter / IPython
await run_server()

# Python
# try:
#     asyncio.run(run_server())
# except KeyboardInterrupt:
#     print(f"gracefully close")

In [ ]:
# Light version
import asyncio, socket

HOST = ""  # Symbolic name meaning all available interfaces
PORT = 50007  # Arbitrary non-privileged port

async def handle_client(reader, writer):
    while True:
        data =  (await reader.read(255))
        print(
            f"raw_data[{data}] size[{len(data) if data else -1}] type[{type(data)}]"
        )
        if (
            (not data)
            or (data == b"\n")
            or (data == b"quit\n")
            or (data == b"quit")
        ):
            # client console type `Enter` to close client
            break

        writer.write(data)
        await writer.drain()
    # print(f"Disconnected by {client.getpeername()}")
    writer.close()

async def run_server():
    server = await asyncio.start_server(handle_client, HOST, PORT)
    async with server:
        await server.serve_forever()

# Jupyter / IPython
await run_server()

## Selector

In [ ]:
import selectors
import socket

HOST = ""  # Symbolic name meaning all available interfaces
PORT = 50007  # Arbitrary non-privileged port

sel = selectors.DefaultSelector()

def accept(sock, mask):
    conn, addr = sock.accept()  # Should be ready
    print('accepted', conn, 'from', addr)
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, read)

def read(conn, mask):
    data = conn.recv(1000)  # Should be ready
    if data and data != b"\n":
        print('echoing', repr(data), 'to', conn)
        conn.send(data)  # Hope it won't block
    else:
        print('closing', conn)
        sel.unregister(conn)
        conn.close()

sock = socket.socket()
sock.bind((HOST, PORT))
sock.listen(100)
sock.setblocking(False)
sel.register(sock, selectors.EVENT_READ, accept)

try:
    while True:
        events = sel.select()
        for key, mask in events:
            print(f"key[{key}] mash[{mask}]")
            callback = key.data
            callback(key.fileobj, mask)
except KeyboardInterrupt:
    print(f"gracefully close")
    sock.close()
    sel.close()

In [ ]:
import selectors
from typing import IO, TextIO, BinaryIO

FILE = "/tmp/a.bin"

sel = selectors.DefaultSelector()

def read(fd: IO, mask):
    print(f"read[{fd.read()}] mask[{mask}]")

fd = open(FILE, "w+")
sel.register(fd, selectors.EVENT_READ, read)

try:
    while True:
        events = sel.select()
        for key, mask in events:
            print(f"key[{key}] mash[{mask}]")
            callback = key.data
            callback(key.fileobj, mask)
except KeyboardInterrupt:
    print(f"gracefully close")
    fd.close()
    sel.close()

# Client 1
# 
# > hello mom 1
# > quit

## Troubleshooting

Send TCP Data by Console

```sh
# keyword acts as stdin
nc 127.0.0.1 50007
# more debug info
nc 127.0.0.1 50007 -d
# one request
echo "hello echo server" | nc 127.0.0.1 50007
curl telnet://127.0.0.1:50007 <<< someText
```

View Socket State

```sh
netstat -an | grep 50007

watch -n 3 "netstat -an | grep 50007"
```


